In [1]:
import warnings
warnings.filterwarnings("ignore")
import MDAnalysis as mda
import nglview as nv
from nglview.datafiles import PDB
u = mda.Universe(PDB)
elements = mda.topology.guessers.guess_types(
    u.atoms.names)
u.add_TopologyAttr('elements', elements)
lig = u.select_atoms("resname LRT")
nv.show_mdanalysis(lig)

NGLWidget()

In [2]:
import nglview as nv
import numpy as np
view = nv.show_mdanalysis(u)
view.add_unitcell()
view.add_representation("ball+stick")
view.control.rotate(
mda.lib.transformations.quaternion_from_euler(
-np.pi/2, np.pi/3, np.pi/6, 'rzyz').tolist())
view.control.zoom(-0.3);view

NGLWidget()

In [3]:
view = nv.show_mdanalysis(lig);view

NGLWidget()

In [4]:
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import GRO
u = mda.Universe(GRO)
sol = u.select_atoms('resname SOL')
ow_hw1 = sol.select_atoms('name OW or name HW1').split('residue')
ow_hw2 = sol.select_atoms('name OW or name HW2').split('residue')
ow_mw = sol.select_atoms('name OW or name MW').split('residue')
u.add_bonds(ow_hw1 + ow_hw2 + ow_mw)

In [5]:
import MDAnalysis as mda
import nglview as nv
from nglview.datafiles import PDB
u = mda.Universe(PDB)
view = nv.show_mdanalysis(u)
view.add_representation('point','resname LRT')
view

NGLWidget()

In [6]:
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import GRO
u = mda.Universe(GRO)
sol = u.select_atoms('resname SOL')
ow_hw1 = sol.select_atoms('name OW or name HW1').split('residue')
ow_hw2 = sol.select_atoms('name OW or name HW2').split('residue')
ow_mw = sol.select_atoms('name OW or name MW').split('residue')
u.add_bonds(ow_hw1 + ow_hw2 + ow_mw)

In [7]:
#nv.show_mdanalysis(sol)

In [8]:
u = mda.Universe.empty(6, 2, atom_resindex=[
    0, 0, 0, 1, 1, 1], trajectory=True)

In [9]:
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import PDB_small
import nglview as nv
import numpy as np
from IPython.core.display import Image
print("Using MDAnalysis version", mda.__version__)
print("Using NGLView version", nv.__version__)

Using MDAnalysis version 1.0.0
Using NGLView version 2.7.7


In [10]:
n_residues = 1000
n_atoms = n_residues * 3
# create resindex list
resindices = np.repeat(range(n_residues), 3)
assert len(resindices) == n_atoms
print("resindices:", resindices[:10])
# all water molecules belong to 1 segment
segindices = [0] * n_residues
print("segindices:", segindices[:10])

resindices: [0 0 0 1 1 1 2 2 2 3]
segindices: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [11]:
# create the Universe
sol = mda.Universe.empty(n_atoms,
                         n_residues=n_residues,
                         atom_resindex=resindices,
                         residue_segindex=segindices,
                         trajectory=True) # necessary for adding coordinates
sol

<Universe with 3000 atoms>

In [12]:
sol.add_TopologyAttr('name', ['O', 'H1', 'H2']*n_residues)
sol.atoms.names

array(['O', 'H1', 'H2', ..., 'O', 'H1', 'H2'], dtype=object)

In [13]:
sol.add_TopologyAttr('type', ['O', 'H', 'H']*n_residues)
sol.atoms.types

array(['O', 'H', 'H', ..., 'O', 'H', 'H'], dtype=object)

In [14]:
sol.add_TopologyAttr('resname', ['SOL']*n_residues)
sol.atoms.resnames

array(['SOL', 'SOL', 'SOL', ..., 'SOL', 'SOL', 'SOL'], dtype=object)

In [15]:
sol.add_TopologyAttr('resid', list(range(1, n_residues+1)))
sol.atoms.resids

array([   1,    1,    1, ..., 1000, 1000, 1000])

In [16]:
#view = nv.show_mdanalysis(sol);view

In [17]:
sol.add_TopologyAttr('segid', ['SOL'])
sol.atoms.segids

array(['SOL', 'SOL', 'SOL', ..., 'SOL', 'SOL', 'SOL'], dtype=object)

In [18]:
h2o = np.array([[ 0,        0,       0      ], 
                [ 0.95908, -0.02691, 0.03231],
                [-0.28004, -0.58767, 0.70556]])

In [19]:
grid_size = 10
spacing = 8

coordinates = []

# translating h2o coordinates around a grid
for i in range(n_residues):
    x = spacing * (i % grid_size)
    y = spacing * ((i // grid_size) % grid_size)
    z = spacing * (i // (grid_size * grid_size))

    xyz = np.array([x, y, z])

    coordinates.extend(h2o + xyz.T)

print(coordinates[:10])

[array([0., 0., 0.]), array([ 0.95908, -0.02691,  0.03231]), array([-0.28004, -0.58767,  0.70556]), array([8., 0., 0.]), array([ 8.95908, -0.02691,  0.03231]), array([ 7.71996, -0.58767,  0.70556]), array([16.,  0.,  0.]), array([16.95908, -0.02691,  0.03231]), array([15.71996, -0.58767,  0.70556]), array([24.,  0.,  0.])]


In [20]:
coord_array = np.array(coordinates)
assert coord_array.shape == (n_atoms, 3)
sol.atoms.positions = coord_array

In [21]:
sol_view = nv.show_mdanalysis(sol)
sol_view.add_representation('ball+stick', selection='all')
sol_view.center()
sol_view

NGLWidget()

In [22]:
assert not hasattr(sol, 'bonds')

In [23]:
bonds = []
for o in range(0, n_atoms, 3):
    bonds.extend([(o, o+1), (o, o+2)])
bonds[:10]

[(0, 1),
 (0, 2),
 (3, 4),
 (3, 5),
 (6, 7),
 (6, 8),
 (9, 10),
 (9, 11),
 (12, 13),
 (12, 14)]

In [24]:
sol.add_TopologyAttr('bonds', bonds)
sol.bonds

<TopologyGroup containing 2000 bonds>

In [25]:
print(sol.atoms[0].bonds)
print(sol.atoms[-10:].bonds)

<TopologyGroup containing 2 bonds>
<TopologyGroup containing 7 bonds>


In [26]:
protein = mda.Universe(PDB_small)
nv.show_mdanalysis(protein)

NGLWidget()

In [27]:
cog = sol.atoms.center_of_geometry()
print('Original solvent center of geometry: ', 
      cog)
cog2 = sol.atoms.center_of_geometry()
print('New solvent center of geometry: ', cog2)

Original solvent center of geometry:  [36.22634681 35.79514029 36.24595657]
New solvent center of geometry:  [36.22634681 35.79514029 36.24595657]


In [28]:
cog = protein.atoms.center_of_geometry()
print('Original solvent center of geometry: ',
cog);protein.atoms.positions -= cog
cog2 = protein.atoms.center_of_geometry()
print('New solvent center of geometry: ',cog2)

Original solvent center of geometry:  [-3.66508082  9.60502842 14.33355791]
New solvent center of geometry:  [8.30580288e-08 3.49225059e-08 2.51332265e-08]


In [29]:
combined = mda.Merge(protein.atoms, sol.atoms)
combined_view = nv.show_mdanalysis(combined)
combined_view.add_representation("ball+stick", selection="not protein")
combined_view

NGLWidget()

In [30]:
no_overlap = combined.select_atoms("same resid \
as (not around 6 protein)")

In [31]:
u = mda.Merge(no_overlap)
view = nv.show_mdanalysis(u)
view.add_representation("ball+stick", selection="not protein")
view

NGLWidget()

In [32]:
u.segments.segids

array(['4AKE', 'SOL'], dtype=object)

In [33]:
core_segment = u.add_Segment(segid='CORE')
core_segment.atoms

<AtomGroup with 0 atoms>

In [34]:
core_atoms = u.select_atoms(
'resid 1:29 or resid 60:121 or resid 160-214')
core_atoms.residues.segments = core_segment
core_segment.atoms

<AtomGroup with 2744 atoms>

In [35]:
nmp_segment = u.add_Segment(segid='NMP')
lid_segment = u.add_Segment(segid='LID')
nmp_atoms = u.select_atoms('resid 30:59')
nmp_atoms.residues.segments = nmp_segment
lid_atoms = u.select_atoms('resid 122:159')
lid_atoms.residues.segments = lid_segment


In [36]:
domain_view = nv.show_mdanalysis(u)
domain_view.color_by('chainID')
domain_view

NGLWidget()